In [55]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import matplotlib.pyplot as plt
import sys
import csv

from pathlib import Path
current_path = Path().absolute()
parent_path = os.path.dirname(current_path)
if parent_path not in sys.path:
    sys.path.append(parent_path)
from aeronetlib import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
file = r'D:\AERONET\data\20210101_20211231_Toulon.lev20'


def angstrom_func(x1,y1,a,x):
    return y1*(x/x1)**(-a)


ant = Aeronet(file)    
# print(aods.ds.keys())

ds1 = ant.extract_from_datetime(2021,8,15,11,26,52)
keynames = ds1.keys()
for key in keynames:
    if 'Angstrom' in key:
        print(key,ds1[key])

wls,aods = ant.extract_from_datetime_effective_aods(2021,8,15,11,26,52)  
print(wls,aods)


# plt.figure()
# plt.plot(wls,vals,'o')
# for i in range(len(wls)):
#     plt.plot(wls,angstrom_func(wls[i],vals[i],ang,wls),'--',label=str(wls[i]))

# plt.plot(wls,A1*wls**(-a1),'r',label='llsq')
# plt.plot(wls,angstrom_func(550,aod550,a1,wls),'b',label='angstrom law')
# plt.legend()
# plt.show()

440-870_Angstrom_Exponent [0.438927]
380-500_Angstrom_Exponent [0.604746]
440-675_Angstrom_Exponent [0.509401]
500-870_Angstrom_Exponent [0.423214]
340-440_Angstrom_Exponent [0.586538]
440-675_Angstrom_Exponent[Polar] [-999.]
[0.3408 0.3811 0.4394 0.5005 0.6745 0.8708 1.0213 1.6422] [0.411284 0.394106 0.354959 0.334407 0.285784 0.26497  0.247857 0.193558]


In [39]:
def calculate_angstrom([wl1,wl2],(aod1,aod2)):
    # calculate the angstrom coef alpha from two aods 
    # aod1 = aod2*(wl1/wl1)**(-alpha)
    return -1*(np.log(aod1/aod2))/(np.log(wl1/wl2))

SyntaxError: invalid syntax (1936397130.py, line 1)

In [71]:
from aeronetlib import *
from scipy.stats import linregress

ant = Aeronet(file)
# ds2 = ant.extract_from_datetime(2021,8,15,11,26,52)
wls,vals = ant.extract_from_datetime_effective_aods(2021,8,15,11,26,52) 

wl550 = 550
if np.min(wls)<1:
    wl550 = 0.55
wl440 = 440
wl675 = 675
if np.min(wls)<1:
    wl440 = 0.44
    wl675 = 0.675

difs440 = np.abs(wls-wl440)
difs675 = np.abs(wls-wl675)
ind440 = np.where(difs440==np.min(difs440))[0][0]
ind675 = np.where(difs675==np.min(difs675))[0][0]

aods_log = np.log(aods)
wls_log = np.log(wls)

cov = np.cov(wls_log,aods_log)
sxy = cov[0,1]
sxx = cov[0,0]
alpha = -sxy/sxx
beta = np.exp(np.mean(aods_log)-alpha*np.mean(wls_log))
aod550 = beta*wl550**(-alpha)
print('sr',alpha,beta,aod550)

p = np.polyfit(wls_log,aods_log,1)
alpha = -p[0]
beta = np.exp(p[1])
aod550 = beta*wl550**(-alpha)
print('polyfit',alpha,beta,aod550)

initial_guess = [1.0,np.exp(np.mean(aods_log))]
params,covariance = curve_fit(angstrom_formula_2,wls,aods,p0=initial_guess)
alpha,beta = params
aod550 = beta*wl550**(-alpha)
print('curve_fit',alpha,beta,aod550)    

slope,intercept,r_value,p_value,std_err = linregress(wls_log,aods_log)

alpha = -slope
beta = np.exp(intercept)
aod550 = beta*wl550**(-alpha)
print('linregress',alpha,beta,aod550)

alpha = calculate_angstrom(wls[[ind440,ind675]],aods[[ind440,ind675]])
aod550 = aods[ind440]*(wl550/wls[ind440])**(-alpha)
print('440-675nm',alpha,aod550)
# alpha,beta = fit_llsq(wls,vals)
# print(alpha,beta)


# print('llsq',calculate_aod550(wls,vals,method='llsq'))
# print('440-675nm',calculate_aod550(wls,vals,method='440-675nm'))

sr 0.46973726210605676 0.3729156818149202 0.4938237419729408
polyfit 0.46973726210605615 0.24519024605582143 0.3246867072290211
curve_fit 0.47635302582362926 0.24430927312519124 0.3248022058726882
linregress 0.4697372621060567 0.24519024605582132 0.32468670722902104
440-675nm 0.5057990537753718 0.31685564640901437
